In [9]:
import numpy as np

## Viscosity Functions
From "Fundamentals of Machine Component Design" Section 12, pp 633-635
Reference TBL12-1

Note: 

(F-32)*5/9 = C

6.89 * microreyn = milli-PascalSec  --> 6.89 * microreyn /1000 = PascalSec --> 6.89 * 1000 reyn = PascalSec

In [11]:
# Idek what the fuck that scaling is on the plot. 
# Its like exponential scaling, which I have never even seen before.
# It shows empiral formulas for the basic SAE oils tho
def Visc_SAE_Seireg(Temp, Grade, Metric = False):
    if (Metric):
        Temp = (Temp * 9/5) + 32 # C to F
    match Grade:
        case 10:
            m0 = 0.0158E-6
            b = 1157.5
        case 20:
            m0 = 0.0136E-6
            b = 1271.6
        case 30:
            m0 = 0.0141E-6
            b = 1360.0
        case 40:
            m0 = 0.0121E-6
            b = 1474.4
        case 50:
            m0 = 0.0170E-6
            b = 1509.6
        case 60:
            m0 = 0.0187E-6
            b = 1564.0
    if (not Metric):
        return m0 * np.e**( b / (Temp + 95))    
    else:
        return m0 * np.e**( b / (Temp + 95)) * 6.89 * 1000 # convert back to Metric

# https://wiki.anton-paar.com/us-en/engine-oil/
def Visc_SAE_Inter(Temp, Grade, Metric = False):
    match Grade:
        case "10W-40":
            if (Metric): mu = 6.61 * np.e**(-0.0484 * Temp)
            else: mu = 2.27E-3 * np.e**(-0.0269 * Temp)
        case "30":
            if (Metric): mu = 18 * np.e**(-0.0708 * Temp)
            else: mu = 9.18E-3 * np.e**(-0.0393 * Temp)
    return mu

            

In [18]:
def Sommerfeld(mu, r, c, N, p):
    S = (r/c)**2 * mu * N / p 
    print("Sommerfeld #: " + str(S))
    return S 

def TempDelta(Tval, p, iter = 0):
    delT = Tval * p / 9.7 # for Imperial
    if iter != 0:
        print("Temperature Delta (" + str(iter) + "): " + str(delT))    
    return delT

def TempFactor(S, l, d):
    if(abs(l - d) < .001):
        factT = .349109 + 6.00940 * S + .047467 * S**2
    elif(abs(l/d - .5) < .001):
        factT = .394552 + 6.392527 * S - .036013 * S**2
    elif(abs(l/d - .25) < .001):
        factT = .933828 + 6.437512 * S - .011048 * S**2
    else: 
        factT = NULL
    print("Temp Factor: " + str(factT))
    return(factT)

def convertTemp(Temp, Metric = False):
    if (Metric):
        return (Temp * 9/5) + 32
    else:
        return (Temp - 32) * 5/9


# EMAE370 Chapter 12: Journal Bearings

## Iterative Approach

In [13]:
Grd = 30 #SAE30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 4
r_in = 4

p_psi = load_lbf / (2*r_in * l_in)

c = r_in*2/1000

rps = 900 / 60

print("Pressure (psi): " + str(p_psi))

Pressure (psi): 312.5


In [14]:
# Guess Temperature
Tout_g = 205
Tavg_g = (Tin_degF + Tout_g)/2
print("Guess Film Average Temp, degF: " + str(Tavg_g))

Guess Film Average Temp, degF: 192.5


In [ ]:
def NaturalJournal(Grade, Metric = False, Tout_guess, T_in, load, rpm, Tin, l, r, c):
    p = load / (2*r *l)
    rps = rpm /60 
    
    

In [17]:
Tout = Tout_g

error = 1.1 # throwaway value
firstIterFlag = True
convergence_res = .1 # deg F
n = 1
while error > convergence_res:
    if firstIterFlag:   
        Tavg = (Tin_degF + Tout)/2
        mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
        print("Viscosity: " + str(mu))
        S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
        delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)
        firstIterFlag = False
    
    n = n+1
    # Storing for comparison to exit while loop
    Tavg_prev = Tavg
    mu_prev = mu 
    S_prev = S 
    delT_prev = delT 

    Tout = Tin_degF + delT 
    Tavg = Tin_degF + delT/2
    mu = Visc_SAE_Seireg(Temp = Tavg, Grade = Grd, Metric = False)
    S = Sommerfeld(mu = mu, r = r_in, c = c, N = rps, p = p_psi)
    delT = TempDelta(Tval = TempFactor(S = S, l = l_in, d = 2*r_in), p = p_psi, iter = n)

    error = abs(delT - delT_prev) 
    if (error <= convergence_res):
        print("S ~ last iteration Sommerfeld: " + str(S) + '\n' + "Tout ~ last iteration Temp: " + str(Tin_degF + delT))
        output = (S, Tin_degF + delT)



Viscosity: 1.5981621311834866e-06
Sommerfeld #: 0.01917794557420184
Temp Factor: 0.5171342895368269
Temperature Delta (1): 16.66025417322252
Sommerfeld #: 0.02056067943306376
Temp Factor: 0.5259714742231698
Temperature Delta (2): 16.944957288117585
Sommerfeld #: 0.02051117870129257
Temp Factor: 0.525655112676366
Temperature Delta (3): 16.93476522797571
S ~ last iteration Sommerfeld: 0.02051117870129257
Tout ~ last iteration Temp: 196.9347652279757


## Manual Approach

In [22]:
## SAE 30

load_lbf = 10E3
rpm = 900
Tin_degF = 180

l_in = 4
r_in = 4

# --- #

p_psi = load_lbf / (2*r_in * l_in)

c = .008

rps = 900 / 60

print("Pressure (psi): " + str(p_psi))

Pressure (psi): 312.5


In [23]:
Tout_1 = 205
Tavg_1 = (Tin_degF + Tout_1)/2
print("Guess Film Average Temp, degF: " + str(Tavg_1))

Guess Film Average Temp, degF: 192.5


In [24]:
mu_1 = 1.4E-6

In [25]:
S_1 = Sommerfeld(mu = mu_1, r = r_in, c = c, N = rps, p = p_psi)

Sommerfeld #: 0.0168


In [26]:
delT_1 = TempDelta(Tval = TempFactor(S = S_1, l = l_in, d = 2*r_in), p = p_psi, iter = 1)

T_2 = Tin_degF + delT_1
print(T_2)

Temp Factor: 0.50193628929088
Temperature Delta (1): 16.170627876639177
196.17062787663917


In [27]:
Tavg_2 =  Tin_degF + delT_1/2
print("Guess Film Average Temp, degF: " + str(Tavg_2))

Guess Film Average Temp, degF: 98.08531393831959


In [28]:
mu_2 = 1.8E-6
S_2 = Sommerfeld(mu = mu_2, r = r_in, c = c, N = rps, p = p_psi)
delT_2 = TempDelta(Tval = TempFactor(S = S_2, l = l_in, d = 2*r_in), p = p_psi, iter = 2)
T_3 = Tin_degF + delT_2
print(T_3)

Sommerfeld #: 0.021599999999999998
Temp Factor: 0.53261378097472
Temperature Delta (2): 17.158949129340208
197.15894912934021


In [29]:
Tavg_3 =  Tin_degF + delT_2/2
print("Guess Film Average Temp, degF: " + str(Tavg_3))

Guess Film Average Temp, degF: 188.5794745646701


In [30]:
mu_3 = 1.8E-6
S_3 = Sommerfeld(mu = mu_3, r = r_in, c = c, N = rps, p = p_psi)
delT_3 = TempDelta(Tval = TempFactor(S = S_3, l = l_in, d = 2*r_in), p = p_psi, iter = 3)
T_4 = Tin_degF + delT_3

Sommerfeld #: 0.021599999999999998
Temp Factor: 0.53261378097472
Temperature Delta (3): 17.158949129340208


In [31]:
Tout = Tin_degF + delT_3
print(Tout)

197.15894912934021


In [32]:
h_0 = c * .8
print("h_0: " + str(h_0))
f = c/r_in * 1.4
print("f: " + str(f))
Q = 5.85 * r_in * c * rps * l_in
print("Qfull: " + str(Q))
Qs = .94 * Q 
print("Side Flow Qs: " + str(Qs))


h_0: 0.0064
f: 0.0028
Qfull: 11.232000000000001
Side Flow Qs: 10.55808
